In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np


from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils

import tensorflow as tf

2023-02-21 22:42:49.987616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('Bank_Customers.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [4]:
# df.info()

In [5]:
df=df.drop(columns=['RowNumber','CustomerId','Surname'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [6]:
X = df.drop(columns='Exited',axis=1)
y = df['Exited']

In [7]:
# LabelEncoder for Gender
label_encoder = preprocessing.LabelEncoder()
X['Gender']= label_encoder.fit_transform(X['Gender'])
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.80,3,1,0,113931.57
3,699,France,0,39,1,0.00,2,0,0,93826.63
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64
9996,516,France,1,35,10,57369.61,1,1,1,101699.77
9997,709,France,0,36,7,0.00,1,0,1,42085.58
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52


In [8]:
# OneHotEncoder for Geography
X = pd.get_dummies(X, columns = ['Geography'])
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,1,0,0
9996,516,1,35,10,57369.61,1,1,1,101699.77,1,0,0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,0,0
9998,772,1,42,3,75075.31,2,1,0,92888.52,0,1,0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

In [10]:
sc = preprocessing.StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
X_train

array([[-0.11883785,  0.92691317, -0.4710917 , ..., -1.0100505 ,
        -0.57869742,  1.75961989],
       [-0.76899335,  0.92691317,  0.57933321, ..., -1.0100505 ,
        -0.57869742,  1.75961989],
       [ 1.08859379, -1.07884971, -1.04405074, ...,  0.9900495 ,
        -0.57869742, -0.56830456],
       ...,
       [-0.20139728,  0.92691317, -0.56658487, ...,  0.9900495 ,
        -0.57869742, -0.56830456],
       [-0.14979763, -1.07884971, -0.94855756, ..., -1.0100505 ,
         1.72801876, -0.56830456],
       [-1.03731149, -1.07884971,  0.10186734, ...,  0.9900495 ,
        -0.57869742, -0.56830456]])

In [12]:
X_test

array([[-1.26434992, -1.07884971, -0.28010535, ...,  0.9900495 ,
        -0.57869742, -0.56830456],
       [ 0.05660094,  0.92691317,  0.77031956, ..., -1.0100505 ,
        -0.57869742,  1.75961989],
       [ 0.33523901, -1.07884971, -1.23503708, ...,  0.9900495 ,
        -0.57869742, -0.56830456],
       ...,
       [ 0.20107994, -1.07884971,  1.72525129, ..., -1.0100505 ,
        -0.57869742,  1.75961989],
       [-0.5006752 , -1.07884971, -1.61700978, ..., -1.0100505 ,
        -0.57869742,  1.75961989],
       [ 0.97507458, -1.07884971, -0.75757122, ..., -1.0100505 ,
        -0.57869742,  1.75961989]])

In [13]:
ann = tf.keras.models.Sequential()

2023-02-21 22:43:00.294599: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
ann.add(tf.keras.layers.Dense(units= 6, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units= 6, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units= 1, activation = 'sigmoid'))

In [15]:
ann.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [21]:
ann.fit(x=X_train, y=y_train, batch_size= 150, epochs=15,verbose= 1)

Epoch 1/15
54/54 [==============================] - 0s 3ms/step - loss: 0.3458 - accuracy: 0.8550
Epoch 2/15
54/54 [==============================] - 0s 3ms/step - loss: 0.3458 - accuracy: 0.8541
Epoch 3/15
54/54 [==============================] - 0s 3ms/step - loss: 0.3457 - accuracy: 0.8546
Epoch 4/15
54/54 [==============================] - 0s 3ms/step - loss: 0.3456 - accuracy: 0.8549
Epoch 5/15
54/54 [==============================] - 0s 3ms/step - loss: 0.3454 - accuracy: 0.8547
Epoch 6/15
54/54 [==============================] - 0s 3ms/step - loss: 0.3452 - accuracy: 0.8551
Epoch 7/15
54/54 [==============================] - 0s 3ms/step - loss: 0.3453 - accuracy: 0.8545
Epoch 8/15
54/54 [==============================] - 0s 3ms/step - loss: 0.3450 - accuracy: 0.8551
Epoch 9/15
54/54 [==============================] - 0s 3ms/step - loss: 0.3447 - accuracy: 0.8553
Epoch 10/15
54/54 [==============================] - 0s 3ms/step - loss: 0.3447 - accuracy: 0.8544
Epoch 11/15
54/54 [